In [1]:
MARKET = 'NSE'

In [2]:
import sys
import pathlib
import numpy as np
import pandas as pd
import yaml
import asyncio

from ib_insync import IB, util, Option, MarketOrder, Contract
from typing import Callable, Coroutine, Union

In [3]:
# Specific to Jupyter. Will be ignored in IDE / command-lines
import IPython as ipy
if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    import nest_asyncio
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None
    
    THIS_FOLDER = '' # Dummy for jupyter notebook's current folder
    BAR_FORMAT = "{l_bar}{bar:-20}{r_bar}"

In [4]:
# Get capability to import programs from `asyncib` folder
cwd = pathlib.Path.cwd() # working directory from where python was initiated
DATAPATH = cwd.joinpath('data', MARKET.lower()) # path to store data files
LOGFILE = cwd.joinpath(THIS_FOLDER, 'data', 'log', 'temp.log') # path to store log files

IBPATH = cwd.parent.parent.joinpath('asyncib') # where ib programs are stored

# append IBPATH to import programs.
if str(IBPATH) not in sys.path:  # Convert it to string!
    sys.path.append(str(IBPATH))
    
IBDATAPATH = IBPATH.joinpath('data', MARKET.lower())

In [ ]:
# Get the host, port, cid
from engine import Vars

ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID

In [ ]:
# Get the pickle files
from os import listdir
fs = listdir(DATAPATH)

files = [f for f in fs if f[-4:] == '.pkl']
for f in files:
    exec(f"{f.split('.')[0]} = pd.read_pickle(DATAPATH.joinpath(f))")
np.sort(np.array(files))

In [13]:
cwd = pathlib.Path.cwd() # working directory from where python was initiated
DATAPATH = cwd.joinpath('data', MARKET.lower()) # path to store data files
TEMPL_PATH = pathlib.Path.cwd().joinpath(THIS_FOLDER, "data", "template")
P = dict()
for f in ['df_opt_prices.pkl', 'z_temp_df_opt_prices.pkl']:
    try:
        p[str(f)] = pd.read_pickle(DATAPATH.joinpath(f))
    except FileNotFoundError:
        p[str(f)] = pd.read_pickle(TEMPL_PATH.joinpath('df_price.pkl'))

ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

In [10]:
DATAPATH

WindowsPath('C:/Users/User/Documents/Business/Projects/learn/ib/data/nse')

In [ ]:
# * IMPORTS
import os
from ib_insync import util
from support import Vars, Timer, get_dte, calcsdmult_df, get_prob, get_prec, fallrise
from engine import get_opts, get_unds, executeAsync, price, margin, post_df

In [ ]:
# * FUNCTION INPUTS
MARKET = MARKET
EARLIEST = False # Earliest expiries are useful for aggressive Friday sessions
SAVE = True

In [ ]:
# * SETUP

ibp = Vars(MARKET.upper())  # IB Parameters from var.yml

HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID
# THIS_FOLDER = os.path.dirname(os.path.abspath(__file__))

LOGPATH = pathlib.Path.cwd().joinpath(THIS_FOLDER, "data", "log")
DATAPATH = pathlib.Path.cwd().joinpath(THIS_FOLDER, "data", MARKET.lower())

# * SETUP LOGS AND CLEAR THEM
LOGFILE = LOGPATH.joinpath(MARKET.lower() + "_nakeds.log")
util.logToFile(path=LOGFILE, level=30)
with open(LOGFILE, "w"):
    pass

# . start the time
nakeds_time = Timer("nakeds")
nakeds_time.start()

In [ ]:
# * CODE

# Get the options
df_opts = get_opts(MARKET)

In [ ]:
# Get the unds

# ... load the symlots
df_symlots = pd.read_pickle(DATAPATH.joinpath('df_symlots.pkl'))

# ... Get und_cts from the symlots for df_opts.symbol
und_cts = df_symlots[df_symlots.symbol.isin\
                      (df_opts.symbol.unique())]\
                            .contract.drop_duplicates()

# ... Get the underlyings
df_unds = get_unds(MARKET=MARKET,
                   und_cts = und_cts,
                   SAVE = True,
                   RUN_ON_PAPER = False)

In [ ]:
# * RECALCULATE UNDERLYING sdMult AND ITS PROBABILITIES

# ... remove excess fields

cols1 = ['symbol', 'strike', 'expiry', 'right']
cols2 = cols1 + ['conId', 'contract', 'lot']

df1 = df_opts[cols2]

dte  = df1.expiry.apply(get_dte).reset_index(drop=True)

# ... process dtes
df1 = df1.assign(dte=dte)

df1 = df1[df1.dte > 0].reset_index(drop=True)  # Remove negative dtes

# ... make 0 dte positive to avoid sqrt errors
df1.loc[df1.dte == 0, "dte"] = 1

In [ ]:
if EARLIEST: # keep only the earliest expiring dte for each symbol
    df1 = df1[df1.dte == df1.groupby('symbol').dte.transform(min)]

In [ ]:
# ... integrate iv and undPrice
df1["iv"] = df1.symbol.map(
    df_unds.set_index("symbol").iv.to_dict())
df1["undPrice"] = df1.symbol.map(
    df_unds.set_index("symbol").undPrice.to_dict())

# ... calculate sdMult and probablities
df1["sdMult"] = calcsdmult_df(df1.strike, df1)
df1["prob"] = df1.sdMult.apply(get_prob)

# ... rename iv to und_iv
df1.rename(columns={'iv': 'und_iv'}, inplace=True)

In [ ]:
# Cleanup df_opts

# ... remove options without time value
m1 = ((df1.right == 'C') & (df1.strike > df1.undPrice)) | \
    ((df1.right == 'P') & (df1.strike < df1.undPrice))

# ... remove out-of-fence (sdMult) options
m2 = ((df1.right == 'C') & (df1.sdMult > ibp.CALLSTDMULT)) | \
    ((df1.right == 'P') & (df1.sdMult > ibp.PUTSTDMULT))

df2 = df1[m1&m2].reset_index(drop=True)

In [ ]:
# Get the price and margins

# ... build contract and order

opt_cons = df2.contract
opt_ords = [MarketOrder("SELL", lot / lot)
            if MARKET.upper() == "SNP"
            else MarketOrder("SELL", lot)
            for lot in df2.lot]

# ... get option price and iv
with IB().connect(HOST, PORT, CID) as ib:
    df_opt_prices = ib.run(
        executeAsync(
            ib=ib,
            algo=price,
            cts=opt_cons,
            CONCURRENT=40,
            TIMEOUT=8,
            post_process=post_df,
            DATAPATH=DATAPATH,
            **{"FILL_DELAY": 5.5},
            OP_FILENAME="",
            SHOW_TQDM=True,
        )
    )

    # to prevent first TimeoutError()
    ib.disconnect()
    IB().waitOnUpdate(timeout=ibp.FIRST_XN_TIMEOUT)

In [ ]:
# ... get option margins
opt_cos = [(c, o) for c, o in zip(opt_cons, opt_ords)]

with IB().connect(HOST, PORT, CID) as ib:
    df_opt_margins = ib.run(
        executeAsync(
            ib=ib,
            algo=margin,
            cts=opt_cos,
            CONCURRENT=200,
            TIMEOUT=5.5,
            post_process=post_df,
            DATAPATH=DATAPATH,
            OP_FILENAME="",
            SHOW_TQDM=True,
            **{"FILL_DELAY": 5.5},
        )
    )

In [ ]:
# * GET OPTION sdMult AND ITS PROBABILITIES

# ... integrate undPrice and und_iv to opt price
df_pr = df_opt_prices.set_index('conId')\
                     .join(df2.set_index('conId')[['undPrice', 'und_iv']])\
                     .reset_index()

# ... replace opt_iv with und_iv where unavailable
df_pr = df_pr.assign(iv=df_pr.iv.combine_first(df_pr.und_iv))

# ... calculate opt sdMult and probablities
df_pr["dte"] = df_pr.expiry.apply(get_dte)
df_pr["opt_sdMult"] = calcsdmult_df(df_pr.strike, df_pr)
df_pr["opt_prob"] = df_pr.opt_sdMult.apply(get_prob)
df_pr.rename(columns={'iv': 'opt_iv'}, inplace=True)

# ... integrate df_pr to df_opts
cols3 = ['conId', 'bid', 'ask', 'close', 'last', 'price', 'opt_sdMult', 'opt_prob']
df3 = df2.set_index('conId').join(df_pr[cols3].set_index('conId')).reset_index()

# ... add intrinsic and time values
df3 = df3.assign(intrinsic=np.where(
    df3.right == "C",
    (df3.undPrice - df3.strike).clip(0, None),
    (df3.strike - df3.undPrice).clip(0, None),
))
df3 = df3.assign(timevalue=df3.price - df3.intrinsic)

# Get return-on-margin (rom)

# ... integrate opt price to opt margins
df_m = df_opt_margins.set_index('conId').join(df3.set_index('conId')[['lot', 'price', 'dte', 'timevalue']]).reset_index()

# ... compute rom based on timevalue
df_m["rom"] = (
    (df_m.timevalue * df_m.lot - df_m.comm).clip(0) /
    df_m.margin * 365 / df_m.dte)

# ... integrate margin and rom
df4 = df3.set_index('conId').join(df_m[['conId', 'margin', 'rom']].set_index('conId')).reset_index()

df4 = df4[df4.rom > 0]\
    .sort_values("rom", ascending=False)\
    .reset_index(drop=True)

In [ ]:
# DETERMINE THE BEST ROM FOR RIGHT, DTE

# ... sort
df5 = df4.sort_values(['symbol', 'dte', 'right', 'sdMult'], 
                          ascending=[True, True, False, True])

In [ ]:
# ... get the max rom
max_rom = df5.groupby(['symbol', 'dte', 'right']).rom.transform(max)
df5['max_rom'] = max_rom

In [ ]:
# .establish expRom
#    ... for those whose RoM is < MINROM, make it equal to MINROM
df4["expRom"] = np.maximum(ibp.MINEXPROM, df4.rom)

# . set expPrice to be based on expRom
df4["expPrice"] = (df4.expRom \
                    * np.maximum(ibp.MINOPTSELLPRICE, df4.price) \
                    / df4.rom).apply(lambda x: get_prec(x, ibp.PREC))

# . remove NaN from expPrice
df4 = df4.dropna(subset=["expPrice"]).reset_index(drop=True)

In [ ]:
# * PICKLE AND SAVE TO EXCEL

# Checkpoint setup
if SAVE:
    
    if EARLIEST:
        OP_FILENAME = 'df_nakeds.pkl'
    else:
        OP_FILENAME = 'df_earliest.pkl'
        
    XL_FILENAME = OP_FILENAME.split('.')[0]+'.xlsx'

    df4.to_pickle(DATAPATH.joinpath(OP_FILENAME))

    df_calls = df4[df4.right == "C"].reset_index(drop=True)
    df_puts = df4[df4.right == "P"].reset_index(drop=True)

    # ... initiate Excel writer object
    writer = pd.ExcelWriter(DATAPATH.joinpath(XL_FILENAME), engine="xlsxwriter")

    cols = ['contract', 'conId', 'symbol', 'strike', 'expiry', 'right', 'lot', 'dte',
           'und_iv', 'undPrice', 'sdMult', 'prob', 'opt_sdMult', 'intrinsic', 'timevalue', 'margin',
            'bid', 'ask', 'close', 'last', 'price', 'expPrice', 'rom', 'max_rom', 'expRom', 'opt_prob']

    df4[cols].to_excel(
        writer, sheet_name="All", float_format="%.2f", index=False, freeze_panes=(1, 1)
    )

    # ... split calls and puts
    df_calls[cols].to_excel(
        writer,
        sheet_name="Calls",
        float_format="%.2f",
        index=False,
        freeze_panes=(1, 1),
    )

    df_puts[cols].to_excel(
        writer, sheet_name="Puts", float_format="%.2f", index=False, freeze_panes=(1, 1)
    )

    # ... prepare sheet objects
    all_sheet = writer.sheets["All"]
    puts_sheet = writer.sheets["Calls"]
    calls_sheet = writer.sheets["Puts"]
    sheets = [all_sheet, puts_sheet, calls_sheet]

    for sht in sheets:
        # Hide all rows without data
        sht.set_default_row(hide_unused_rows=True)

        sht.set_column("A:B", None, None, {"hidden": True})  # Hide contract and conId

    # save
    try:
        writer.save()
    except Exception as e:
        print(f"\nError {e}: {XL_FILENAME} is open or has some issues!!!\n")

nakeds_time.stop()